In [1]:
import pandas as pd
import json
from sqlalchemy import create_engine

# Koneksi ke database
db_url = "mysql+pymysql://root:@localhost/keranjang_homemade"
engine = create_engine(db_url)

# Query untuk mendapatkan daftar tahun
query_years_bar = """
SELECT DISTINCT YEAR(order_date) AS year
FROM (
    SELECT order_date FROM orders WHERE status = 'Completed'
    UNION ALL
    SELECT order_date FROM offline_orders WHERE status = 'Completed'
) AS combined_orders
ORDER BY year DESC;
"""

# Query untuk penjualan online berdasarkan tahun
query_online_bar = """
SELECT 
    YEAR(o.order_date) AS year,
    p.product_name,
    SUM(oi.quantity) AS online_quantity
FROM products p
LEFT JOIN order_items oi ON p.product_id = oi.product_id
LEFT JOIN orders o ON oi.order_id = o.order_id
WHERE o.status = 'Completed'
GROUP BY year, p.product_name
"""

# Query untuk penjualan offline berdasarkan tahun
query_offline_bar = """
SELECT 
    YEAR(oo.order_date) AS year,
    p.product_name,
    SUM(ooi.quantity) AS offline_quantity
FROM products p
LEFT JOIN offline_order_items ooi ON p.product_id = ooi.product_id
LEFT JOIN offline_orders oo ON ooi.offline_order_id = oo.offline_order_id
WHERE oo.status = 'Completed'
GROUP BY year, p.product_name
"""

# Ambil data dari database
df_years = pd.read_sql(query_years_bar, engine)
years = df_years['year'].tolist()

df_online_bar = pd.read_sql(query_online_bar, engine)
df_offline_bar = pd.read_sql(query_offline_bar, engine)

# Gabungkan data online dan offline berdasarkan produk dan tahun
df_bar = pd.merge(
    df_online_bar, df_offline_bar, on=['year', 'product_name'], how='outer'
).fillna(0)

# Format data per tahun
data_per_year = {}
for year in years:
    year_data = df_bar[df_bar['year'] == year]
    data_per_year[year] = {
        "products": year_data["product_name"].tolist(),
        "online_sales_quantities": year_data["online_quantity"].astype(int).tolist(),
        "offline_sales_quantities": year_data["offline_quantity"].astype(int).tolist(),
    }

# Simpan semua data ke file JSON
data_bar = {
    "years": years,
    "data": data_per_year
}

with open('C:\\laragon\\www\\keranjang_homemade\\graphics\\top_products.json', 'w') as f:
    json.dump(data_bar, f)